# Introduction

This is an example of a Jupyter notebook where you can intermingle text with code.  A notebook is made up of a series of cells.  This first cell is an example of a markdown cell containing text. [Markdown](https://www.markdownguide.org/cheat-sheet/) provides a simple system for formatting text (adding headings, inserting links, using bold and underline, etc.). You can double click on this cell to see the underlaying markdown.

The next cell (below) is a code cell.  You can use lots of different languages for the code cells.  We will be using Python3. To execute a cell (to run the code inside of it) you press SHIFT-ENTER. You can also use the same command on markdown cells to render the markdown.

Typically the first code cell in a notebook imports all of the necessary python libraries. 

In [2]:
import aitk.robots
aitk.robots.__version__

'0.9.43'

### Devyani Mahajan
- CPSC021
- CPSC035
- CPSC063
- CPSC013

###  Exercise 1
Insert a new cell above this one (by using the icon with a plus sign in the top right of this cell).  Then designate that cell to be *Markdown* (by toggling the *Code* designation to *Markdown* at the top of this notebook).  

Then add your name as a heading. Under that heading list all of the CS courses you've taken so far at the College. Use a bulleted list.

## Create a world

A world is a rectangular area with a given width and height that may contain walls, bulbs, food, and robots. We will create a simple world with a bulb in the bottom right corner and a small wall next to it.

In [13]:
simple_world = aitk.robots.World(width=200, height=200, scale=2.0)
simple_world.add_bulb("yellow", 180, 180, 0, 100)
simple_world.add_wall("blue", 150, 200, 155, 150)

Random seed set to: 6678281


## Create a robot

A robot can sense and act in the world.  A robot may have:
- RangeSensors that return distances to obstacles in cm
- LightSensors that return a brightness value in the range [0, 1]; light is blocked by walls
- SmellSensors that return an odor value in the range [0, 1]; odor spreads around walls
- Cameras that return images that include the walls, bulbs, food, and other robots in the world

Below we create a robot with one RangeSensor in the center front and two LightSensors on the front left and right.

In [14]:
robot = aitk.robots.Scribbler(x=20, y=180, a=45, max_trace_length=60)
robot.add_device(aitk.robots.RangeSensor(position=(6,0),max=20,a=0,width=57.3,name="f-ir"))
robot.add_device(aitk.robots.LightSensor(position=(6,-5),name="left-light"))
robot.add_device(aitk.robots.LightSensor(position=(6,5),name="right-light"))

## Add the robot to the world
Once you have both a world and a robot, add the robot to the world, update the world, and save the world. Now you are ready to start writing a controller for the robot.

In [15]:
simple_world.add_robot(robot)
simple_world.update()
simple_world.save()

## Watch the world
It is helpful to be able to watch the robot moving around the world in order to debug your controller.

In [16]:
simple_world.watch()

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x0…

## Create a controller
A controller is a function that takes a robot as its only parameter. It returns True to end the simulation. It should check the state of the robot's sensors to choose an appropriate action. 

NOTE: It should **not** include loops because the underlying simulator will repeatedly execute the controller multiple times per second.

Consult the aitk.robots cheat sheet for how to access the state of the robot's sensors and to make the robot move.

Below is a simple controller meant to move the robot around the world, avoiding walls, and stopping when the light source is found. Notice that rather than using print statements to help with debugging, it is more fruitful to have the robot speak information to indicate what state it is in. 

In [33]:
def avoid_walls(robot):
    if robot["left-light"].get_brightness() + robot["right-light"].get_brightness() > 1.9:
        # found exit
        robot.speak("Found light!")
        return True
    elif robot["left-light"].get_brightness() > robot["right-light"].get_brightness():
        robot.speak("Turn left")
        robot.move(0.5, +0.1)
    elif robot["left-light"].get_brightness() < robot["right-light"].get_brightness(): 
        robot.speak("Turn right")
        robot.move(0.5, -0.1)
    elif robot["f-ir"].get_distance() < robot["f-ir"].get_max():
        # avoid wall ahead
        robot.speak("Avoid")
        robot.move(0.1, -0.3)
    else:
        robot.speak("Forward")
        robot.speak("hello %.1f %.1f" % (robot["left-light"].get_brightness(), robot["right-light"].get_brightness()))
        robot.move(1, 0)

## Run the simulator
Before starting a run, it is helpful to reset the world back to its orginal starting conditions. One way to run the simulator is to have the controller run indefinitely until True is returned. Because there could be multiple robots in the simulator, the run method expects to receive a **list** of controllers, one per robot.  We only have a single robot in our environment, so our list is of length one.

In [34]:
simple_world.reset()
simple_world.run([avoid_walls])

Using random seed: 6678281


0it [00:00, ?it/s]

Simulation stopped at: 00:00:16.90; speed 0.98 x real time


True

## Editing and re-running
The safest way to re-run all of the code after you've made a change is to go to the *Kernel* menu at the top of the notebook and choose *Restart and Run All*. If your notebook ever ends up in a strange state, this is a good way to reset everything.  

Or you can use SHIFT-ENTER to re-run a cell after you've made a change.  For example, suppose you change the `avoid_walls` controller and you want to test the new version.  Do SHIFT-ENTER in that cell and in the next code cell that resets and runs the simulator.  Watch the outcome of your new controller in the watch window. 

### Exercise 2
Modify the `avoid_walls` controller so that the robot speaks it's current light readings when going forward. The speak command expects a single string as the parameter.  You can use string formatting to accomplish this. 

For example, if you had two float variables `x` and `y`, then you could have the robot speak their current values like this: 

`robot.speak("hello %.1f %.1f" % (x, y))`  

Be sure to re-test your controller after every change.

### Exercise 3
Modify the `avoid_walls` controller so that it will turn towards the light source.  

If the left light reading is stronger than the right one, then it should move forward left. And if the right light reading is stronger than the left one, then it should move forward right. Make sure that for each case the robot speaks something unique that let's you know what it's doing. 

Once you make this change, the robot should find the light source faster than it did with the original version of the controller.